## Step 1: Install Required Dependencies

In [ ]:
# Install required packages for TFLite conversion
import subprocess
import sys

print("Installing required packages...")
subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "tensorflow", "tensorflow-hub"])
print("✓ Dependencies installed successfully")

## Step 2: Import Libraries

In [ ]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO
import tensorflow as tf

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Step 3: Define Conversion Parameters

In [ ]:
# Define model locations and output directory
models_to_convert = {
    "yolov8n": "/Users/kevinjones/Documents/paraglider_detection/paraglider_recognition-8/yolov8n.pt",
    "yolov8s": "/Users/kevinjones/Documents/paraglider_detection/training/yolov8s.pt",
    "yolo11n": "/Users/kevinjones/Documents/paraglider_detection/training/yolo11n.pt",
}

output_dir = "/Users/kevinjones/Documents/paraglider_detection/tflite_models"
os.makedirs(output_dir, exist_ok=True)

print(f"Output directory: {output_dir}")
print(f"\nModels to convert:")
for model_name, model_path in models_to_convert.items():
    exists = "✓" if os.path.exists(model_path) else "✗"
    print(f"  {exists} {model_name}: {model_path}")

## Step 4: Convert Models

**Note**: This step may take several minutes depending on your hardware. Conversion time increases with model size.

In [ ]:
def convert_model_to_tflite(model_name: str, model_path: str, output_dir: str) -> str:
    """
    Convert a YOLOv8 model to TFLite format.
    
    Args:
        model_name: Name of the model (e.g., 'yolov8n')
        model_path: Full path to the .pt model file
        output_dir: Directory to save the .tflite model
        
    Returns:
        Path to the converted .tflite model
    """
    try:
        print(f"\n{'='*60}")
        print(f"Converting {model_name}...")
        print(f"{'='*60}")
        
        # Check if model exists
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Model not found: {model_path}")
        
        print(f"Loading model from: {model_path}")
        model = YOLO(model_path)
        
        print(f"Exporting to TFLite format...")
        # Export to TFLite format
        export_result = model.export(
            format="tflite",        # Export to TFLite format
            imgsz=640,              # Input image size
            half=False,             # Use full precision
            device=0 if tf.config.list_physical_devices('GPU') else "cpu",
        )
        
        # Get the exported model path
        tflite_filename = f"{model_name}.tflite"
        output_path = os.path.join(output_dir, tflite_filename)
        
        # If conversion succeeded, show results
        if os.path.exists(export_result):
            shutil.copy(export_result, output_path)
            file_size_mb = os.path.getsize(output_path) / 1024 / 1024
            print(f"\n✓ Successfully converted {model_name}!")
            print(f"  Output: {output_path}")
            print(f"  File size: {file_size_mb:.2f} MB")
            return output_path
        else:
            print(f"✓ Conversion completed (check ultralytics directory)")
            return export_result
            
    except Exception as e:
        print(f"✗ Error converting {model_name}: {str(e)}")
        return None

# Convert all models
conversion_results = {}
for model_name, model_path in models_to_convert.items():
    result = convert_model_to_tflite(model_name, model_path, output_dir)
    conversion_results[model_name] = result

## Step 5: Verify Converted Models

In [ ]:
import os

print(f"{'='*60}")
print("CONVERSION SUMMARY")
print(f"{'='*60}")

print(f"\nConverted models in {output_dir}:")
if os.path.exists(output_dir):
    files = os.listdir(output_dir)
    if files:
        for filename in sorted(files):
            filepath = os.path.join(output_dir, filename)
            size_mb = os.path.getsize(filepath) / 1024 / 1024
            print(f"  ✓ {filename} ({size_mb:.2f} MB)")
    else:
        print("  No files found")
else:
    print(f"  Directory not found: {output_dir}")

print(f"\n{'='*60}")

## Step 6: Model Comparison

Compare the original .pt models with the converted .tflite models

In [ ]:
import pandas as pd

# Create comparison table
comparison_data = []

for model_name, model_path in models_to_convert.items():
    if os.path.exists(model_path):
        pt_size_mb = os.path.getsize(model_path) / 1024 / 1024
        
        tflite_path = os.path.join(output_dir, f"{model_name}.tflite")
        if os.path.exists(tflite_path):
            tflite_size_mb = os.path.getsize(tflite_path) / 1024 / 1024
            reduction = ((pt_size_mb - tflite_size_mb) / pt_size_mb) * 100
            
            comparison_data.append({
                "Model": model_name,
                ".pt Size (MB)": f"{pt_size_mb:.2f}",
                ".tflite Size (MB)": f"{tflite_size_mb:.2f}",
                "Size Reduction": f"{reduction:.1f}%"
            })

if comparison_data:
    df = pd.DataFrame(comparison_data)
    print("\nModel Size Comparison:")
    print(df.to_string(index=False))
else:
    print("No completed conversions to compare")

## Using the TFLite Models

### On Mobile (Android/iOS)
```python
from tflite_runtime.interpreter import Interpreter

# Load the TFLite model
interpreter = Interpreter(model_path="yolov8n.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
```

### On Jetson Device
```bash
# Install TFLite runtime
pip install tflite-runtime

# Run inference
python jetson/inference_tflite.py
```

### For Deployment
1. Copy the `.tflite` file to your target device
2. Use TensorFlow Lite Interpreter or supported framework
3. Ensure input image size is 640x640 pixels
4. Process model outputs using the standard YOLOv8 format

In [3]:
import os
import shutil
from pathlib import Path
from ultralytics import YOLO
import tensorflow as tf

In [4]:
model = YOLO('yolov8s.pt')

# Export the model to TFLite format
# This command generates a file like 'your_model_name_float32.tflite' in your directory
model.export(format="tflite")

Ultralytics 8.3.234 🚀 Python-3.12.11 torch-2.9.1 CPU (Apple M4)
Ultralytics 8.3.234 🚀 Python-3.12.11 torch-2.9.1 CPU (Apple M4)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs

PyTorch: starting from 'yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (21.5 MB)

PyTorch: starting from 'yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (21.5 MB)
requirements: Ultralytics requirements ['tf_keras<=2.19.0', 'sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0,<1.4.0', 'onnx>=1.12.0,<=1.19.1', 'onnx2tf>=1.26.3', 'onnxslim>=0.1.71'] not found, attempting AutoUpdate...
requirements: Ultralytics requirements ['tf_keras<=2.19.0', 'sng4onnx>=1.0.1', 'onnx_graphsurgeon>=0.3.26', 'ai-edge-litert>=1.2.0,<1.4.0', 'onnx>=1.12.0,<=1.19.1', 'onnx2tf>=1.26.3', 'onnxslim>=0.1.71'] not found, attemptin

I0000 00:00:1764827606.511925  838648 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1764827606.512285  838648 single_machine.cc:376] Starting new session
W0000 00:00:1764827607.098735  838648 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1764827607.098751  838648 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
I0000 00:00:1764827607.702072  838648 devices.cc:76] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
I0000 00:00:1764827607.702135  838648 single_machine.cc:376] Starting new session
W0000 00:00:1764827607.098735  838648 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1764827607.098751  838648 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
I0000 00:00:1764827607.702072  838648 devices.cc:76] Number of

TensorFlow SavedModel: export success ✅ 44.0s, saved as 'yolov8s_saved_model' (107.2 MB)

TensorFlow Lite: starting export with tensorflow 2.20.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov8s_saved_model/yolov8s_float32.tflite' (42.8 MB)

Export complete (44.2s)
Results saved to /Users/kevinjones/Documents/paraglider_detection
Predict:         yolo predict task=detect model=yolov8s_saved_model/yolov8s_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=yolov8s_saved_model/yolov8s_float32.tflite imgsz=640 data=coco.yaml  
Visualize:       https://netron.app

TensorFlow Lite: starting export with tensorflow 2.20.0...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov8s_saved_model/yolov8s_float32.tflite' (42.8 MB)

Export complete (44.2s)
Results saved to /Users/kevinjones/Documents/paraglider_detection
Predict:         yolo predict task=detect model=yolov8s_saved_model/yolov8s_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=y

W0000 00:00:1764827608.470313  838648 tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
W0000 00:00:1764827608.470328  838648 tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.


'yolov8s_saved_model/yolov8s_float32.tflite'